<a href="https://colab.research.google.com/github/shubhamksingh1/WordTagging/blob/main/SimilarityMatrix/Similarity_Demo_Data_Overlap_Webex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Import Statements

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib as plt
import sklearn

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Reading the cleaned file of Incident description

In [ ]:
df_desc = pd.read_csv("/content/drive/My Drive/NetworkIncident/df_desc_clean.csv")

In [ ]:
df_desc.describe()

,Description,Subcategory
count,7086,7086
unique,7086,9
top,seem internet issues today slow seems ongoing ...,lan
freq,1,3012


In [ ]:
df_desc.shape

(7086, 2)

In [ ]:
df_desc.head()

,Description,Subcategory
0,impossible connect ssid lh eu mobile user conn...,wifi
1,establishment local printer network laboratory...,lan
2,tools cannot opened due lock data protection s...,lan
3,user called unable access internet say message...,lan
4,frbl lp access lafarge applications frbl lp ac...,lan


In [ ]:
df_desc.tail()

,Description,Subcategory
7081,pb connection wifi cable internet network pb c...,wifi
7082,guests cannot connect wi fi guest house cannot...,wifi
7083,la fare les oliviers site granulate connection...,wifi
7084,user trying test vpn see wifi connections asse...,wifi
7085,connect network holcim public internet access ...,wifi


In [ ]:
print('\nFrequency distribution of unique values => \n\n'+ str(df_desc['Subcategory'].value_counts(dropna=False).sort_values(ascending = False)))


Frequency distribution of unique values => 

lan                         3012
proxy pac                   1306
internet                     925
wifi                         848
other                        633
remote access                203
wan                          157
browser                        1
laptops desktops tablets       1
Name: Subcategory, dtype: int64


In [ ]:
d1 = df_desc.groupby(df_desc['Subcategory'])['Description'].apply(', '.join).reset_index()

In [ ]:
d1

,Subcategory,Description
0,browser,access mail slow access mail slow
1,internet,contact wifi northend weeks whole site cable w...
2,lan,establishment local printer network laboratory...
3,laptops desktops tablets,mr schneider always receives error message wan...
4,other,"cleanup disk cleanup disk, issue vpn issue vpn..."
5,proxy pac,"internet access internet access, hello everyon..."
6,remote access,external user conmm access via vpn external us...
7,wan,"level inspection unloading, hello coming back..."
8,wifi,impossible connect ssid lh eu mobile user conn...


In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
ss = SnowballStemmer("english", ignore_stopwords=True)
stop = set(stopwords.words("english"))

In [ ]:
def cleaning_text(inp):
    corpus = []
    reply = re.sub('[^a-zA-Z0-9]', ' ', str(inp))
    reply = reply.lower()
    reply = reply.split()
    reply = [ss.stem(word) for word in reply if not word in set(stop)]
    reply = list(set(reply))
    return reply

In [ ]:
d1['Description'] = d1['Description'].apply(lambda x: cleaning_text(x))

In [ ]:
d1

,Subcategory,Description
0,browser,"[mail, access, slow]"
1,internet,"[purley, everyon, almost, unplug, count, throu..."
2,lan,"[francoi, bbrc, wellan, aarschot, xp, shala, l..."
3,laptops desktops tablets,"[pleas, otherwis, made, error, messag, noteboo..."
4,other,"[david, isdn, unplug, everyon, bousaada, count..."
5,proxy pac,"[everyon, scope, almost, triton, difficulti, u..."
6,remote access,"[rheinzabern, qvp, sand, keep, entgeltabrechnu..."
7,wan,"[sunil, unplug, adress, win, creation, keep, a..."
8,wifi,"[purley, everyon, almost, freder, hoilday, dos..."


In [ ]:
d1['Subcategory'].values

array(['browser', 'internet', 'lan', 'laptops desktops tablets', 'other',
       'proxy pac', 'remote access', 'wan', 'wifi'], dtype=object)

In [ ]:
df = pd.DataFrame(index=d1['Subcategory'].values, columns=d1['Subcategory'].values)
df = df.fillna(0)

In [ ]:
df

,browser,internet,lan,laptops desktops tablets,other,proxy pac,remote access,wan,wifi
browser,0,0,0,0,0,0,0,0,0
internet,0,0,0,0,0,0,0,0,0
lan,0,0,0,0,0,0,0,0,0
laptops desktops tablets,0,0,0,0,0,0,0,0,0
other,0,0,0,0,0,0,0,0,0
proxy pac,0,0,0,0,0,0,0,0,0
remote access,0,0,0,0,0,0,0,0,0
wan,0,0,0,0,0,0,0,0,0
wifi,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(len(d1['Subcategory'])):
    for j in range(len(d1['Subcategory'])):
        len_i = len(d1['Description'][i])
        set_A = set(d1['Description'][i])
        set_B = set(d1['Description'][j])
        common = len(set_A&set_B)
        similarity = common/len_i
        similarity = similarity * 100
        if df[d1['Subcategory'][j]][d1['Subcategory'][i]] == 0:
            df[d1['Subcategory'][j]][d1['Subcategory'][i]] = similarity

In [ ]:
df

,browser,internet,lan,laptops desktops tablets,other,proxy pac,remote access,wan,wifi
browser,100,100,100,33,100,100,66,100,100
internet,0,100,67,1,25,24,17,26,42
lan,0,31,100,0,14,13,9,15,26
laptops desktops tablets,3,90,96,100,75,72,72,78,90
other,0,54,67,1,100,42,25,38,49
proxy pac,0,51,59,1,41,100,24,34,44
remote access,0,57,70,2,41,40,100,39,54
wan,0,58,73,2,40,36,25,100,50
wifi,0,54,73,1,29,27,20,29,100


In [ ]:
df.to_csv('/content/drive/My Drive/Webex_Demo/NetworkIncident/desc_clean_similarity.csv',index = False)

### Below Cells just explain how to calculate the one particular value of the loop

In [ ]:
cat1list = list(d1[d1['Subcategory']=="browser"]['Description'])[0]

In [ ]:
cat1list

['mail', 'access', 'slow']

In [ ]:
cat2list = list(d1[d1['Subcategory']=="internet"]['Description'])[0]

In [ ]:
cat1set = set(cat1list)

In [ ]:
cat2set = set(cat2list)

In [ ]:
len(cat1set),len(cat2set)

(3, 2788)

In [ ]:
len(cat1set & cat2set)

3

In [ ]:
common_ele = cat1set & cat2set

In [ ]:
common_ele

{'access', 'mail', 'slow'}

In [ ]:
3 /3 * 100

100.0

### Cosine Similarities between category

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def compute_similarity(str1,str2):
    train = [str1,str2]
    #Define a Count Vectorizer Object
    vector = TfidfVectorizer()

    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = vector.fit_transform(train)

    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [ ]:
df_cos_sim = pd.DataFrame(index=d1['Subcategory'].values, columns=d1['Subcategory'].values)
df_cos_sim = df_cos_sim.fillna(0)

In [ ]:
df_cos_sim

,browser,internet,lan,laptops desktops tablets,other,proxy pac,remote access,wan,wifi
browser,0,0,0,0,0,0,0,0,0
internet,0,0,0,0,0,0,0,0,0
lan,0,0,0,0,0,0,0,0,0
laptops desktops tablets,0,0,0,0,0,0,0,0,0
other,0,0,0,0,0,0,0,0,0
proxy pac,0,0,0,0,0,0,0,0,0
remote access,0,0,0,0,0,0,0,0,0
wan,0,0,0,0,0,0,0,0,0
wifi,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(len(d1['Subcategory'])):
    for j in range(len(d1['Subcategory'])):
        set_A = set(d1['Description'][i])
        set_B = set(d1['Description'][j])
        catA_string=' '.join(set_A) 
        catB_string=' '.join(set_B) 
        similarity = compute_similarity(catA_string,catB_string)[0][1]
        similarity = similarity * 100
        if df_cos_sim[d1['Subcategory'][j]][d1['Subcategory'][i]] == 0:
            df_cos_sim[d1['Subcategory'][j]][d1['Subcategory'][i]] = similarity

In [ ]:
df_cos_sim

,browser,internet,lan,laptops desktops tablets,other,proxy pac,remote access,wan,wifi
browser,100,2,1,5,3,3,2,3,2
internet,2,100,30,6,23,22,19,25,31
lan,1,30,100,4,19,17,16,21,30
laptops desktops tablets,5,6,4,100,7,7,9,8,7
other,3,23,19,7,99,26,19,24,24
proxy pac,3,22,17,7,26,99,18,21,21
remote access,2,19,16,9,19,18,99,18,20
wan,3,25,21,8,24,21,18,100,24
wifi,2,31,30,7,24,21,20,24,99


In [ ]:
df_cos_sim.to_csv('/content/drive/My Drive/Webex_Demo/NetworkIncident/desc_clean_cosine_similarity.csv',index = False)

### Below Cells just explain how to calculate the one particular value of the loop

In [ ]:
cat1_string=' '.join(cat1set) 
cat2_string=' '.join(cat2set) 

In [ ]:
cat1_string,cat2_string

('mail access slow',
 'purley unplug everyon almost count throughout cci straight task ntmynyisimv azur avcet held cdw recap ratho anomali rglen intel dec custom icar detail templat rooney lh leighton lfmobil resolv huriel tvxdnbuuwtjjvnsisinzlcii break millisecond addr point ccallag franck xl silo villag optim ls imag host bruno fs moston inspect brows ntag develop factori way regist oper mechan run weather stead pb concern zs free disabl okriftel kirchheim hitchin tech maghul access northern feedback eit sever control cdl human vdz carri murtagh txkwmfpqstjmv ashaka nfcmall enter darlington silk enquir yes fact orang rmjzbujimm duntilland youtub counti deleg project probabl direct u credenti thingworx explor age beddinghoven ciscovpn half croy ciz notifi ata instead plan ceas karlovac cox www postbox postag soc domain cycl sap bt tlp com googledr eb recept welsh subject northend progress solin republ thank prior approx mikhalkov visit iiwibmlpijoibwljcm hq gladbach peterborough worke

In [ ]:
compute_similarity(cat1_string,cat2_string)[0][1]*100

2.341738550893845

### Spacy Similarities between category

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
df_spacy_sim = pd.DataFrame(index=d1['Subcategory'].values, columns=d1['Subcategory'].values)
df_spacy_sim = df_spacy_sim.fillna(0)

In [ ]:
df_spacy_sim


,browser,internet,lan,laptops desktops tablets,other,proxy pac,remote access,wan,wifi
browser,0,0,0,0,0,0,0,0,0
internet,0,0,0,0,0,0,0,0,0
lan,0,0,0,0,0,0,0,0,0
laptops desktops tablets,0,0,0,0,0,0,0,0,0
other,0,0,0,0,0,0,0,0,0
proxy pac,0,0,0,0,0,0,0,0,0
remote access,0,0,0,0,0,0,0,0,0
wan,0,0,0,0,0,0,0,0,0
wifi,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(len(d1['Subcategory'])):
    for j in range(len(d1['Subcategory'])):
        set_A = set(d1['Description'][i])
        set_B = set(d1['Description'][j])
        catA_string=' '.join(set_A) 
        catB_string=' '.join(set_B)
        docA = nlp(catA_string) 
        docB = nlp(catB_string)
        similarity = docA.similarity(docB)
        similarity = similarity * 100
        if df_spacy_sim[d1['Subcategory'][j]][d1['Subcategory'][i]] == 0:
            df_spacy_sim[d1['Subcategory'][j]][d1['Subcategory'][i]] = similarity

/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if

In [ ]:
df_spacy_sim

,browser,internet,lan,laptops desktops tablets,other,proxy pac,remote access,wan,wifi
browser,100,57,54,69,61,60,59,59,57
internet,57,100,99,90,99,99,99,99,99
lan,54,99,100,88,98,99,99,99,99
laptops desktops tablets,69,90,88,100,93,92,92,92,91
other,61,99,98,93,100,99,99,99,99
proxy pac,60,99,99,92,99,100,99,99,99
remote access,59,99,99,92,99,99,100,99,99
wan,59,99,99,92,99,99,99,100,99
wifi,57,99,99,91,99,99,99,99,100
